You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
    verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
        "3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
# its using langchain? lol
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 

Title: Exploring the Latest Trends in Artificial Intelligence

Outline:
I. Introduction
- Definition of Artificial Intelligence
- Importance of AI in today's technology landscape

II. Latest Trends in Artificial Intelligence
- Machine learning advancements
- Natural language processing developments
- AI in healthcare and medicine
- AI ethics and bias considerations

III. Key Players in the AI Industry
- Google
- Amazon
- Microsoft
- IBM
- Tesla

IV. Noteworthy News in Artificial Intelligence


I now can give a great answer

Final Answer:
# Exploring the Latest Trends in Artificial Intelligence

## Introduction

Artificial Intelligence, commonly known as AI, is a rapidly evolving technology that simulates human intelligence processes such as learning, reasoning, and problem-solving. In today's technology landscape, AI plays a crucial role in revolutionizing various industries, driving innovation, and enhancing efficiency. The integration of AI in everyday devices and systems has become increasingly prevalent, shaping the way we interact with technology and the world around us.

## Latest Trends in Artificial Intelligence

### Machine Learning Advancements
One of the most significant trends in AI is the advancements in machine learning algorithms. Machine learning enables systems to learn from data, identify patterns, and make decisions with minimal human intervention. Recent developments in deep learning, a subset of machine learning, have led to breakthroughs in image recogn

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# Exploring the Latest Trends in Artificial Intelligence

## Introduction

Artificial Intelligence, commonly known as AI, is a rapidly evolving technology that simulates human intelligence processes such as learning, reasoning, and problem-solving. In today's technology landscape, AI plays a crucial role in revolutionizing various industries, driving innovation, and enhancing efficiency. The integration of AI in everyday devices and systems has become increasingly prevalent, shaping the way we interact with technology and the world around us.

## Latest Trends in Artificial Intelligence

### Machine Learning Advancements
One of the most significant trends in AI is the advancements in machine learning algorithms. Machine learning enables systems to learn from data, identify patterns, and make decisions with minimal human intervention. Recent developments in deep learning, a subset of machine learning, have led to breakthroughs in image recognition, natural language processing, and autonomous driving technologies.

### Natural Language Processing Developments
Natural language processing (NLP) is another area of AI that has seen rapid progress in recent years. NLP focuses on enabling computers to understand, interpret, and generate human language. Applications of NLP include virtual assistants, language translation tools, and sentiment analysis in social media. The evolution of NLP has transformed how we communicate with machines, making interactions more intuitive and seamless.

### AI in Healthcare and Medicine
AI has made significant strides in the healthcare and medical fields, revolutionizing patient care, diagnostics, and treatment strategies. From predictive analytics to personalized medicine, AI-powered solutions are enhancing healthcare delivery, improving patient outcomes, and driving medical research. The integration of AI in healthcare holds immense potential for addressing complex medical challenges and advancing the quality of care worldwide.

### AI Ethics and Bias Considerations
As AI technologies continue to advance, ethical considerations and bias mitigation have become critical areas of focus. Questions surrounding the ethical use of AI, data privacy, and algorithmic bias have sparked debates and discussions within the industry and academia. It is essential for AI developers and policymakers to address these ethical concerns proactively, ensuring that AI systems are designed and deployed responsibly to benefit society as a whole.

## Key Players in the AI Industry

The AI industry is dominated by key players who are at the forefront of innovation and research in artificial intelligence. Companies such as Google, Amazon, Microsoft, IBM, and Tesla are leading the way in developing cutting-edge AI technologies and applications. From self-driving cars to virtual assistants, these industry giants are shaping the future of AI and driving the evolution of intelligent systems across various sectors.

## Noteworthy News in Artificial Intelligence

### Breakthroughs in AI Research
Recent breakthroughs in AI research have led to significant advancements in areas such as reinforcement learning, generative adversarial networks, and explainable AI. These breakthroughs are pushing the boundaries of what AI can achieve and opening up new possibilities for innovation and discovery.

### AI Applications in Various Industries
AI applications are transforming industries such as finance, retail, transportation, and entertainment, revolutionizing business operations and customer experiences. From predictive analytics to robotic process automation, AI solutions are driving efficiency, productivity, and competitive advantage for organizations worldwide.

### Ethical Debates Surrounding AI Technology
The ethical debates surrounding AI technology are gaining prominence as the impact of AI on society becomes more pronounced. Discussions on AI ethics, bias, accountability, and transparency are shaping the development and deployment of AI systems, ensuring that they align with ethical principles and societal values.

## Target Audience Analysis

- Tech-savvy individuals interested in AI advancements
- Business professionals seeking to implement AI in their operations
- Students studying AI and related fields
- AI enthusiasts looking to stay updated on the latest trends

## SEO Keywords

- Artificial Intelligence trends
- AI industry players
- Latest AI news
- Machine learning advancements
- AI applications

## Resources

1. "The AI Revolution: The Road to Superintelligence" - Wait But Why
2. "Human Compatible: Artificial Intelligence and the Problem of Control" by Stuart Russell
3. "Artificial Unintelligence: How Computers Misunderstand the World" by Meredith Broussard
4. Forbes - AI section
5. MIT Technology Review - AI articles

## Call to Action

Stay informed about the latest trends in Artificial Intelligence by following reputable sources and engaging in discussions with industry experts. Keep exploring the endless possibilities of AI technology and its impact on our society. 

In conclusion, Artificial Intelligence continues to reshape the technological landscape, driving innovation, and transforming industries. By staying informed and actively participating in the discourse surrounding AI advancements, we can harness the potential of AI to create a more intelligent and connected world.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [15]:
topic = "how to buy profitable condos in singapore"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on how to buy profitable condos in singapore.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Content Plan: How to Buy Profitable Condos in Singapore

Introduction:
- Brief overview of the real estate market in Singapore
- Importance of investing in profitable condos
- Purpose of the article

Key Points:
1. Latest Trends:
- Emerging locations for profitable condo investments
- Impact of COVID-19 on the real estate market
- Shift towards sustainable and eco-friendly properties

2. Key Players:
- Top real estate developers in Singapore
- Real estate agencies specializing in condo invest

I now can give a great answer

Final Answer:
# How to Buy Profitable Condos in Singapore

## Introduction
The real estate market in Singapore has been experiencing significant growth in recent years, making it an appealing option for investors interested in purchasing condos. Investing in profitable condos is crucial as they can offer a steady income stream and potential capital appreciation over time. This article aims to provide valuable insights and guidance for potential investors, first-time buyers, and individuals seeking real estate investment opportunities in Singapore.

## Latest Trends
A key trend in the Singapore real estate market is the emergence of new locations for profitable condo investments. Areas like the East Coast, Sentosa, and the Central Business District have witnessed increased demand for condos due to their proximity to amenities and transportation links. Moreover, the COVID-19 pandemic has influenced the market, leading to a preference for sustainable and eco

In [16]:
Markdown(result)

# How to Buy Profitable Condos in Singapore

## Introduction
The real estate market in Singapore has been experiencing significant growth in recent years, making it an appealing option for investors interested in purchasing condos. Investing in profitable condos is crucial as they can offer a steady income stream and potential capital appreciation over time. This article aims to provide valuable insights and guidance for potential investors, first-time buyers, and individuals seeking real estate investment opportunities in Singapore.

## Latest Trends
A key trend in the Singapore real estate market is the emergence of new locations for profitable condo investments. Areas like the East Coast, Sentosa, and the Central Business District have witnessed increased demand for condos due to their proximity to amenities and transportation links. Moreover, the COVID-19 pandemic has influenced the market, leading to a preference for sustainable and eco-friendly properties. Buyers are now more interested in properties that prioritize energy efficiency and green spaces.

## Key Players
When considering buying profitable condos in Singapore, it is essential to look at the top real estate developers in the country. Companies like CapitaLand, City Developments Limited, and Frasers Property are renowned for their quality developments and innovative designs. Real estate agencies specializing in condo investments, such as PropNex Realty and ERA Realty Network, can offer valuable guidance and support throughout the purchasing process. Legal and financial advisors with expertise in property purchases can provide expert advice on navigating the legal and financial aspects of buying a condo in Singapore.

## Noteworthy News
Government regulations on property ownership play a significant role in the Singapore real estate market. Understanding the regulations regarding foreign ownership, stamp duties, and property taxes is crucial for making informed investment decisions. Market predictions and forecasts for condo investments can assist investors in identifying potential opportunities and risks in the market. Success stories of investors who have made profitable investments in the Singapore real estate market can serve as inspiration for those looking to enter the market.

## Conclusion
In conclusion, purchasing profitable condos in Singapore requires careful consideration of the latest trends, key players in the market, and noteworthy news related to property investments. By staying informed and seeking advice from real estate experts, potential investors can make informed decisions that align with their financial goals and investment objectives. Whether you are a first-time buyer, an expat looking to invest in Singaporean real estate, or an experienced investor, the Singapore real estate market provides various opportunities for profitable condo investments. Do not hesitate to reach out to real estate agencies or advisors specializing in condo investments for personalized guidance and support in your property investment journey.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).